In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from geopy.geocoders import Nominatim
import warnings
import requests
from sklearn.preprocessing import OneHotEncoder



ModuleNotFoundError: No module named 'geopy'

In [ ]:
dataframe = pd.read_csv('/Users/munjismac/code/munjik/personal-projects/real_estate_project/RawData.csv')
df2 = pd.read_csv('/Users/munjismac/code/munjik/personal-projects/real_estate_project/Data-(Jan20-June20).csv')

In [ ]:
dataframe = dataframe.drop(columns='Additional Lien')

In [ ]:
dataframe.info()

In [ ]:
df2

In [ ]:
df2.info()

In [ ]:
# concat both dataframes
df = pd.concat([dataframe, df2], ignore_index=True)

In [ ]:
df

<h2>Let's clean up some data </h2> 

In [ ]:
# remove the unwanted columns
df = df.drop(columns=['CASE #', 'Special Master','Postponed', 'Time', 'Postponed','Debtor'])

In [ ]:
df = df.drop(labels=128) #last column was all empty anyways
df.isnull().sum() #check to see how many null values we have in each column

In [ ]:
imputer = SimpleImputer(strategy="most_frequent") # Instanciate a SimpleImputer object with strategy of choice
# impute lender
imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = imputer.transform(df[['ARV']]) # Call the "transform" method on the object

imputer.fit(df[['EQUITY']]) # Call the "fit" method on the object
df['EQUITY'] = imputer.transform(df[['EQUITY']]) # Call the "transform" method on the object


In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
# function to remove puncuation
def remove_dollar(x):
    return x.replace('$','')
def replace_comma(x):
    return x.replace(',','')
def replace_period(x):
    return x.replace('.','')
def replace_paran(x):
    return x.replace("(","").replace(")","")
# reformat from str to float
def currency_to_float(x: str) -> float:
    """Converts US currency ``x`` to float."""
    return float(x)
    
#-> potentially use this function to do all in one functions here
# Convert the currency to float.
# def currency_to_float(x: str) -> float:
#     """Converts US currency ``x`` to float."""
#     return float(x.replace("$", "").replace(",", ""))

In [ ]:
df['Sold'] = df['Sold'].astype('str')
df['Sold'] = df['Sold'].apply(remove_dollar)
df['Sold'] = df['Sold'].apply(replace_comma)

In [ ]:
df[(df.Sold) == 'THEY DONT HAVE ANYTHING ON THIS...he\'ll double check when back at office']
df.iloc[63]['Sold'] = 'NaN'
df['Sold'] = df['Sold'].apply(lambda x: currency_to_float(x))


In [ ]:
# sum should have increased from 92 to 93
df['Sold'].isnull().sum()

In [ ]:
mean_imputer = SimpleImputer(strategy="mean") # Instanciate a SimpleImputer object with strategy of choice
mean_imputer.fit(df[['Sold']]) # Call the "fit" method on the object
df['Sold'] = mean_imputer.transform(df[['Sold']]) # Call the "transform" method on the object

In [ ]:
df['Sold'].isnull().sum() #now should be zero once we fill the values in with imputes

In [ ]:
# First convert to type string -> clean Balance column
df['Balance'] = df['Balance'].astype('str')
df['Balance'] = df['Balance'].apply(replace_comma)
df['Balance'] = df['Balance'].apply(remove_dollar)

# First convert to type string -> clean ARV column
df['ARV'] = df['ARV'].astype('str')
df['ARV'] = df['ARV'].apply(replace_comma)
df['ARV'] = df['ARV'].apply(remove_dollar)
# First convert to type string -> clean Equity column
df['EQUITY'] = df['EQUITY'].astype('str')
df['EQUITY'] = df['EQUITY'].apply(replace_comma)
df['EQUITY'] = df['EQUITY'].apply(remove_dollar)
df['EQUITY'] = df['EQUITY'].apply(replace_paran)

In [ ]:
# FIXING THIS ERROR: ValueError: could not convert string to float: 'COMMERCIAL'
df[(df.ARV) == 'COMMERCIAL'] #find location
df.loc[df.ARV == "COMMERCIAL", "ARV"] = None

In [ ]:
df.isnull().sum() #sum for ARV has gone up to 1 because of code above
mean_imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = mean_imputer.transform(df[['ARV']]) # Call the "transform" method on the object

In [ ]:
mean_imputer.statistics_

In [ ]:
df['Balance'] = df['Balance'].apply(lambda x: currency_to_float(x))
df['ARV'] = df['ARV'].apply(lambda x: currency_to_float(x))
df['EQUITY'] = df['EQUITY'].apply(lambda x: currency_to_float(x))

In [ ]:
# lets convert Date to datetime64
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df

In [ ]:
# function to concatante all coordinates into 1 for lats and longs in next step
def coordinates(dataframe_coord):
    cols = ["Address", "City","State","Zip"]
    df['coordinates'] = df[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    return df

In [ ]:
coordinates(df)

In [ ]:
#Turn our new coordinates column into Lats and Longs
def geolocate(dataframe):
    df_address = dataframe['coordinates']
    lats = []
    lngs = []
    #loop through or dataframe addresses
    for address in df_address:
        API_KEY = 'GGTqpEquAqOeHelGwevtJopD2QhYtUKl'
        base_url = 'http://www.mapquestapi.com/geocoding/v1/address'
        location = address

        #call the api function on all of our addresses
        params = {
            'key': API_KEY,
            'location' : location
        }

        response = requests.get(base_url,params=params).json()
        results = response['results'][0]
        lat = results['locations'][0]['latLng']['lat']
        lng = results['locations'][0]['latLng']['lng']
        lats.append(lat)
        lngs.append(lng)
    df_location = pd.DataFrame(list(zip(lats,lngs)),
                           columns=['location_lats', 'location_lngs'])
    dataframe = pd.concat([dataframe,df_location], axis=1)
    return dataframe
    

In [ ]:
print(df.loc[46])

In [ ]:
df.shape

In [ ]:
#dropping this column with useless info
df = df.drop([df.index[46]])

In [ ]:
#resets index to 0
df = df.reset_index(drop=True)

In [ ]:
df = geolocate(df)

In [ ]:
df

In [ ]:
"""FOR REFERENCE ->
One Hot Encoding:
We use this categorical data encoding technique when the features are nominal(do not have any order)
------------------------------------------------------------------------------------------------
Label Encoding or Ordinal Encoding:
We use this categorical data encoding technique when the categorical feature is ordinal. In this case,
retaining the order is important. Hence encoding should reflect the sequence.
"""

In [ ]:
print(df.Lender.nunique()) #OHE this or see next best step for 33 diff lenders
print(df.Address.nunique()) #--> I'd like to change this to Lat's and Longs for ML model
print(df.City.nunique()) #Encode this ? I think
print(df.State.nunique()) #easy encode all 1's
print(df.Zip.nunique())

In [ ]:
#lets drop address and coordinates since we now have lats and longs
df = df.drop(columns=['Address', 'coordinates'])

In [ ]:
df.info()

In [ ]:
#Change Zip to INT64 instead of current object
df['Zip'] = df['Zip'].astype('int64')

In [ ]:
#begin OHE
ohc = OneHotEncoder()
ohe = ohc.fit_transform(df.Lender.values.reshape(-1,1)).toarray()

dfOneHot = pd.DataFrame(ohe, columns=['Lender_' +str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])
dfh = pd.concat([df, dfOneHot], axis = 1)

In [ ]:
df

In [ ]:
dfOneHot

In [ ]:
dfh

In [ ]:
# starting off by starting to see what is selling above judgement and %%% sold vs Scheduled 

In [ ]:
#1.)if sold is above balanced 
#2.) change the objects to their types
#3.) NaN == 0
#4.) Banks to individual #'s' 1 - 33 top 10 banks
# data[(data.Medal == 'Silver') & (data.Gender == 'Men')]



In [ ]:
#FOR TESTING PURPOSES ONLY

In [ ]:
testing_dataframe = df.drop(columns=['Lender'])

In [ ]:
testing_dataframe

In [ ]:
#extract month feature
months = df.Date.dt.month
#extract day of month feature
day_of_months = df.Date.dt.day

In [ ]:
#make a dataframe with months and day of months 
data_feature = pd.DataFrame({
    'months':months,
    'days': day_of_months
})

In [ ]:
feature = pd.concat([testing_dataframe, data_feature], axis=1)
feature = feature.drop(columns='Date')

In [ ]:
#OHE city, NM, and ZIP.
feature

In [ ]:
ohe = OneHotEncoder()
ohe.fit(feature[['State']]) # Fit encoder
feature['State'] = ohe.transform(feature[['State']]).toarray() # Encode Street

In [ ]:
#use BinaryEncoder for ZIP

<h2>Will work on this once basic model idea is done.. for now still quite of feature engineering to be done </h2>

In [ ]:
ohc = OneHotEncoder()
final_ohe = ohc.fit_transform(feature.City.values.reshape(-1,1)).toarray()
final_dfOneHot = pd.DataFrame(final_ohe, columns=['City_'+str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])


In [ ]:
final_dfOneHot

In [ ]:
feature

In [ ]:
feature = pd.concat([feature, final_dfOneHot], axis=1)

In [ ]:
feature = feature.drop(columns='City')

In [ ]:
feature = feature.drop(columns='Zip')

In [ ]:
feature = feature.drop(columns='State')

<h2>The ML Alog's to use is KNN Regression(k-NN) and Random Forest (RF) Regression </h2>

<p>1.) Will begin with RFF since there's no feature scaling required. KNN and any distance is preferred to same scale. </p>

In [ ]:
#Visualise the data
# correlation 
corr = feature.corr()
top_corr_feat = corr.index
plt.figure(figsize=(16, 10))
sns.heatmap(feature[top_corr_feat].corr(), annot=True, fmt='.0%')

In [ ]:
# X and y features
X = feature.drop(columns='Sold')
y = feature['Sold']

In [ ]:
from sklearn.ensemble import RandomForestRegressor #import library
from sklearn.model_selection import train_test_split

model = RandomForestRegressor() #instantiate our Basic model , no hyper parameters yet 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .3, random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)


In [ ]:
predictions

In [ ]:
sns.distplot(y_test - predictions)

In [ ]:
plt.scatter(y_test,predictions)


In [ ]:
y_test

In [ ]:
predictions

In [ ]:
model.score(X_train,y_train)

In [ ]:
actual_values = y_test
plt.scatter(predictions, actual_values, alpha=.7,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Linear Regression Model')
plt.show()

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, predictions[:126], label="predicted")
plt.title("House test and predicted data")
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()